In [62]:
import pandas as pd
import numpy as np

In [63]:
data = pd.read_csv('../data/processed/data.csv', index_col=0)

In [64]:
data.set_index('datetime', inplace=True)
data[['energy_RE', 'energy_FC']] = data[['energy_RE', 'energy_FC']]*10
data.head()

,load_FC,load_RE,Solar_FC,Wind_offshore_FC,Wind_onshore_FC,DK1_Accepted_Down_Volume_(MW),DK1_Accepted_Up_Volume_(MW),DK1_Imbalance_Price_(EUR),DK2_Accepted_Down_Volume_(MW),DK2_Accepted_Up_Volume_(MW),...,energy_FC_error,lambda_IM_hourly_rollavg,lambda_DA_RE_hourly_rollavg,missing_capacity_RE_hourly_rollavg,diff_markets_RE_hourly_rollavg,abs_diff_markets_RE_hourly_rollavg,load_FC_error_hourly_rollavg,Solar_FC_error_hourly_rollavg,Wind_offshore_FC_error_hourly_rollavg,Wind_onshore_FC_error_hourly_rollavg
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00+01:00,3216.0,3260.0,0.0,839.0,2854.0,2117.0,996.0,28.32,187.0,1163.0,...,0.004902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 01:00:00+01:00,3140.0,3186.0,0.0,955.0,2987.0,2192.0,1002.0,10.07,179.0,1163.0,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 02:00:00+01:00,3051.0,3070.0,0.0,512.0,2817.0,1524.0,1530.0,-4.08,181.0,1163.0,...,-0.223514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 03:00:00+01:00,2951.0,2966.0,0.0,443.0,2729.0,1377.0,1653.0,-9.91,80.0,1169.0,...,-0.059160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 04:00:00+01:00,2896.0,2933.0,0.0,437.0,2667.0,1400.0,1669.0,-7.41,82.0,1169.0,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
# Dummy strategy: bid forecast, settle imbalance in balancing market
dummy = pd.DataFrame(index=data.index)

dummy['lambda_DA_RE'] = data['lambda_DA_RE']
print(dummy.head())
dummy['lambda_IM'] = data['lambda_IM']
dummy['energy_FC'] = data['energy_FC']
dummy['energy_RE'] = data['energy_RE']
dummy['imbalance'] = data['energy_RE'] - data['energy_FC']

dummy['DA_revenue'] = dummy['lambda_DA_RE'] * dummy['energy_FC']
dummy['IM_revenue'] = dummy['imbalance'] * dummy['lambda_IM']
dummy['total_revenue'] = dummy['DA_revenue'] + dummy['IM_revenue']


                           lambda_DA_RE
datetime                               
2019-01-01 00:00:00+01:00         28.32
2019-01-01 01:00:00+01:00         10.07
2019-01-01 02:00:00+01:00         -4.08
2019-01-01 03:00:00+01:00         -9.91
2019-01-01 04:00:00+01:00         -7.41


In [66]:
dummy.describe()

,lambda_DA_RE,lambda_IM,energy_FC,energy_RE,imbalance,DA_revenue,IM_revenue,total_revenue
count,17416.000000,17416.000000,17416.000000,17416.000000,17416.000000,17416.000000,17416.000000,17416.000000
mean,34.128741,35.138543,4.514179,4.531795,0.017616,135.671217,-1.795102,133.876114
std,17.545984,32.977142,3.550017,3.602470,1.400566,136.621776,65.608439,140.153194
min,-48.290000,-131.820000,0.000000,0.000000,-9.656863,-428.290278,-1510.064327,-913.773323
25%,23.140000,17.000000,1.187441,1.127451,-0.604104,26.691212,-15.065253,27.810494
50%,35.240000,33.840000,3.913632,3.725490,0.000000,96.839366,0.000000,94.259819
75%,44.140000,43.800000,8.015558,8.284314,0.686275,210.996354,17.007132,208.440387
max,254.440000,899.000000,10.000000,10.000000,9.901961,2389.600000,1294.291103,2190.466667


In [67]:

dummy.to_csv(f'../results/test/Dummy_model/results.csv')


In [68]:
print(data.columns)

Index(['load_FC', 'load_RE', 'Solar_FC', 'Wind_offshore_FC', 'Wind_onshore_FC',
       'DK1_Accepted_Down_Volume_(MW)', 'DK1_Accepted_Up_Volume_(MW)',
       'DK1_Imbalance_Price_(EUR)', 'DK2_Accepted_Down_Volume_(MW)',
       'DK2_Accepted_Up_Volume_(MW)', 'lambda_IM', 'Biomass_RE',
       'Fossil_Gas_RE', 'Fossil_Hard_coal_RE', 'Fossil_Oil_RE', 'Solar_RE',
       'Waste_RE', 'Wind_offshore_RE', 'Wind_onshore_RE', 'lambda_DA_RE',
       'lambda_DA_FC', 'energy_RE', 'energy_FC', 'missing_capacity_RE',
       'diff_markets_RE', 'abs_diff_markets_RE', 'system_state_RE', 'date',
       'lambda_DA_FC_normalized',
       'lambda_DA_FC_zeta_1.0_mu_-0.37961439398567787',
       'lambda_DA_FC_zeta_1.0_mu_0.02682140396829682',
       'lambda_DA_FC_zeta_1.0_mu_0.3542880027276621',
       'energy_FC_zeta_0.3550017391908838_mu_0.11874412244809437',
       'energy_FC_zeta_0.3550017391908838_mu_0.39136321503461935',
       'energy_FC_zeta_0.3550017391908838_mu_0.8015558154203456',
       'load_FC_er

In [69]:
#filter for after 16.01.2019 and before 21.12.2020
#data = data[(data['date'] > '2019-01-16') & (data['date'] < '2020-12-21')] # Jan testing data
#data = data[data['date'] < '2020-12-06'] # Jan training data
#data = data[data['date'] < '2019-11-26'] # Yannick training data
#data = data[data['date'] > '2020-01-01'] # Yannick testing data

data[['energy_FC', 'energy_RE']] = data[['energy_FC', 'energy_RE']]*10

data['DA_revenue'] = data['lambda_DA_RE'] * data['energy_FC']
data['imbalance'] = (data['energy_RE'] -data['energy_FC'])
data['imbalance_revenue'] = data['imbalance'] * data['lambda_IM']
data['imbalance_value'] = data['imbalance'] * (data['lambda_DA_RE'] - data['lambda_IM'])
data['revenue'] = data['DA_revenue'] + data['imbalance_revenue']
print(data['revenue'].sum())
print(data['imbalance_value'].sum())

23315864.096980657
64496.29517620563


In [70]:
data.describe()

,load_FC,load_RE,Solar_FC,Wind_offshore_FC,Wind_onshore_FC,DK1_Accepted_Down_Volume_(MW),DK1_Accepted_Up_Volume_(MW),DK1_Imbalance_Price_(EUR),DK2_Accepted_Down_Volume_(MW),DK2_Accepted_Up_Volume_(MW),...,abs_diff_markets_RE_hourly_rollavg,load_FC_error_hourly_rollavg,Solar_FC_error_hourly_rollavg,Wind_offshore_FC_error_hourly_rollavg,Wind_onshore_FC_error_hourly_rollavg,DA_revenue,imbalance,imbalance_revenue,imbalance_value,revenue
count,17416.000000,17416.000000,17416.000000,17416.000000,17416.000000,17416.000000,17416.000000,17416.000000,17416.000000,17416.000000,...,17392.000000,17392.000000,17392.000000,17392.000000,17392.000000,17416.000000,17416.000000,17416.000000,17416.000000,17416.000000
mean,3857.455903,3856.221406,125.255282,692.297944,1202.238114,1417.185175,779.531465,31.387280,166.175643,815.908016,...,11.282446,1.191279,-1.146895,9.029215,29.912520,1356.712169,0.176160,-17.951024,3.703278,1338.761145
std,716.613244,713.998014,201.826058,418.298758,909.375151,634.383277,242.087260,31.009788,106.361719,134.870986,...,7.154724,32.321077,24.165180,56.402026,109.499017,1366.217756,14.005663,656.084389,369.265652,1401.531937
min,2332.000000,1693.000000,0.000000,0.000000,14.000000,217.000000,341.000000,-131.820000,0.000000,446.000000,...,0.000000,-160.392857,-147.178571,-162.000000,-2658.000000,-4282.902779,-96.568627,-15100.643268,-10698.125657,-9137.733230
25%,3273.000000,3269.000000,0.000000,323.000000,460.750000,889.750000,586.000000,14.170000,81.000000,693.000000,...,5.869500,-20.500000,-1.321429,-27.700000,-29.233333,266.912118,-6.041036,-150.652527,-11.550079,278.104937
50%,3836.000000,3818.000000,4.000000,669.000000,959.000000,1372.500000,736.000000,32.000000,151.000000,794.000000,...,9.398667,0.361905,0.000000,-4.406897,4.666667,968.393658,0.000000,0.000000,0.000000,942.598191
75%,4428.000000,4420.000000,179.000000,1056.000000,1781.000000,1902.000000,924.250000,41.942500,228.000000,932.000000,...,14.982667,19.166667,2.066667,27.566667,74.866667,2109.963535,6.862745,170.071318,13.191647,2084.403869
max,5776.000000,9618.000000,911.000000,1519.000000,4300.000000,3465.000000,2579.000000,2016.070000,777.000000,1269.000000,...,71.930000,161.750000,123.900000,203.133333,554.321429,23896.000000,99.019608,12942.911025,13592.233230,21904.666667


In [71]:
data[['lambda_DA_RE', 'lambda_IM', 'energy_RE', 'energy_FC', 'imbalance', 'imbalance_revenue', 'DA_revenue', 'revenue']].head(10)

,lambda_DA_RE,lambda_IM,energy_RE,energy_FC,imbalance,imbalance_revenue,DA_revenue,revenue
datetime,,,,,,,,
2019-01-01 00:00:00+01:00,28.32,28.32,99.509804,100.000000,-0.490196,-13.882353,2832.000000,2818.117647
2019-01-01 01:00:00+01:00,10.07,10.07,100.000000,100.000000,0.000000,0.000000,1007.000000,1007.000000
2019-01-01 02:00:00+01:00,-4.08,-4.08,100.000000,77.648557,22.351443,-91.193889,-316.806111,-408.000000
2019-01-01 03:00:00+01:00,-9.91,-9.91,99.509804,93.593781,5.916023,-58.627785,-927.514372,-986.142157
2019-01-01 04:00:00+01:00,-7.41,-7.41,100.000000,100.000000,0.000000,-0.000000,-741.000000,-741.000000
2019-01-01 05:00:00+01:00,-12.55,-12.55,99.509804,98.693219,0.816585,-10.248143,-1238.599897,-1248.848039
2019-01-01 06:00:00+01:00,-17.25,-17.25,99.509804,100.000000,-0.490196,8.455882,-1725.000000,-1716.544118
2019-01-01 07:00:00+01:00,-15.07,-15.07,99.509804,96.478714,3.031090,-45.678523,-1453.934222,-1499.612745
2019-01-01 08:00:00+01:00,-4.93,-4.93,100.000000,100.000000,0.000000,-0.000000,-493.000000,-493.000000
